## 1.闭包和嵌套函数
如果在一个函数的内部还定义了另一个函数(注意: 是定义，不是引用!），这个函数就叫嵌套函数。外部的我们叫它外函数，内部的我们叫他内函数。


In [10]:
# 闭包：通过闭包可以创建一些只有当前函数能访问的变量  可以将一些私有的数据藏到的闭包中
#形成闭包的条件：
#     1.函数嵌套
#     2.将内部函数作为返回值返回
#     3.内部函数必须使用到外部函数的变量

def make_average():
    nums=[]
    def average(n):
        nums.append(n)
        return sum(nums)/len(nums)
    return average

a=make_average()  #a是一个函数，是调用make_average()返回后的average函数，这个函数实在make_average内部定义的，而且这个函数能访问到make_average的变量
a(5)
a(10)
        

7.5

In [4]:
def outer(x):
    a = x

    def inner(y):
        b = y
        print(a+b)

    return inner

f1 = outer(1) # 返回inner函数对象+局部变量1(闭包)
f1(10) # 相当于inner(10)。输出11

# 我们的outer方法返回的只是内函数对象吗? 错。我们的outer函数返回的实际上是一个由inner函数和外部引用变量(a)组成的闭包!
# 一般一个函数运行结束的时候，临时变量会被销毁。但是闭包是一个特别的情况。当外函数发现，自己的临时变量会在将来的内函数中用到，
# 自己在结束的时候，返回内函数的同时，会把外函数的临时变量同内函数绑定在一起。这样即使外函数已经结束了，内函数仍然能够使用外函数的临时变量。
# 这就是闭包的强大之处。


11


## 2.装饰器
由于Python装饰器的工作原理主要依赖于嵌套函数和闭包

In [5]:
import time
def func():
    print("hello")
    time.sleep(1)
    print("world")
func()

hello
world


In [6]:
#原始侵入，篡改原函数
import time
def func():
    startTime = time.time()
    
    print("hello")
    time.sleep(1)
    print("world")
    endTime = time.time()
    
    msecs = (endTime - startTime)*1000
    print("time is %d ms" %msecs)
func()


hello
world
time is 1001 ms


In [7]:
#带有参数的装饰器
import time

def deco(func):
    def wrapper(a,b):
        startTime = time.time()
        func(a,b)
        endTime = time.time()
        msecs = (endTime - startTime)*1000
        print("time is %d ms" % msecs)
    return wrapper


@deco
def func(a,b):
    print("hello，here is a func for add :")
    time.sleep(1)
    print("result is %d" %(a+b))

if __name__ == '__main__':
    f = func
    f(3,4)
    #func()


hello，here is a func for add :
result is 7
time is 1001 ms


In [9]:
# *argss  可以传入任意个参数(以元组的形式返回)
# **kwargs  以键值对字典的形式向函数传入函数（以字典的形式返回）

def test(x,y,*a,**b):
    print(x,y,a,b)

test(1,2,3,4,a=1)

1 2 (3, 4) {'a': 1}


In [8]:
#带有不定参数的装饰器
import time

def deco(func):
    def wrapper(*args, **kwargs):  #(*args  设置传入多个参数)
        startTime = time.time()
        func(*args, **kwargs)
        endTime = time.time()
        msecs = (endTime - startTime)*1000
        print("time is %d ms" %msecs)
    return wrapper


@deco
def func(a,b):
    print("hello，here is a func for add :")
    time.sleep(1)
    print("result is %d" %(a+b))

@deco
def func2(a,b,c):
    print("hello，here is a func for add :")
    time.sleep(1)
    print("result is %d" %(a+b+c))


if __name__ == '__main__':
    f = func
    func2(3,4,5)
    f(3,4)
    #func()


hello，here is a func for add :
result is 12
time is 1002 ms
hello，here is a func for add :
result is 7
time is 1001 ms


In [10]:
#多个装饰器

import time

def deco01(func):
    def wrapper(*args, **kwargs):
        print("this is deco01")
        startTime = time.time()
        func(*args, **kwargs)
        endTime = time.time()
        msecs = (endTime - startTime)*1000
        print("time is %d ms" %msecs)
        print("deco01 end here")
    return wrapper

def deco02(func):
    def wrapper(*args, **kwargs):
        print("this is deco02")
        func(*args, **kwargs)

        print("deco02 end here")
    return wrapper

@deco01
@deco02
def func(a,b):
    print("hello，here is a func for add :")
    time.sleep(1)
    print("result is %d" %(a+b))



if __name__ == '__main__':
    f = func
    f(3,4)
    #func()

'''
this is deco01
this is deco02
hello，here is a func for add :
result is 7
deco02 end here
time is 1003 ms
deco01 end here
'''


this is deco01
this is deco02
hello，here is a func for add :
result is 7
deco02 end here
time is 1000 ms
deco01 end here


'\nthis is deco01\nthis is deco02\nhello，here is a func for add :\nresult is 7\ndeco02 end here\ntime is 1003 ms\ndeco01 end here\n'

In [11]:
def dec1(func):  
    print("1111")  
    def one():  
        print("2222")  
        func()  
        print("3333")  
    return one  
  
def dec2(func):  
    print("aaaa")  
    def two():  
        print("bbbb")  
        func()  
        print("cccc")  
    return two  
 
@dec1  
@dec2  
def test():  
    print("test test")    #多个装饰器执行的顺序就是从最后一个装饰器开始，执行到第一个装饰器，再执行函数本身。
  
test()  


aaaa
1111
2222
bbbb
test test
cccc
3333


In [17]:
class FooParent(object):
    def __init__(self):
        self.parent = 'I\'m the parent.'
        print ('Parent')
    
    def bar(self,message):
        print ("%s from Parent" % message)
 
class FooChild(FooParent):
    def __init__(self):
        # super(FooChild,self) 首先找到 FooChild 的父类（就是类 FooParent），然后把类 FooChild 的对象转换为类 FooParent 的对象
        super(FooChild,self).__init__()    
        print ('Child')
        
    def bar(self,message):
        super().bar(message)
        print ('Child bar fuction')
        print (self.parent)
 
if __name__ == '__main__':
    fooChild = FooChild()
    fooChild.bar('HelloWorld')

Parent
Child
HelloWorld from Parent
Child bar fuction
I'm the parent.


## 3.迭代器、生成器、可迭代对象的介绍

In [2]:
list=[1,2,3,4]
it = iter(list)# 创建迭代器对象
# print(next(it))
# print(next(it))
# print(next(it))
for x in it:
    print (x, end=" ")

1 2 3 4 

In [4]:
# 字符串，列表或元组对象都可用于创建迭代器：
#  1）迭代器类型的定义：
#       当类中定义了 __iter__ 和 __next__ 两个方法
#       __iter__方法需要返回对象本身，即self
#       __next__ 方法，返回下一个数据，如果没有数据，则需要抛出一个 StopIteration的异常  （Iteration翻译为迭代）
    
    
#创建一个迭代器
class MyNumbers:
    def __iter__(self):
        self.a = 1
        return self
 
    def __next__(self):
        x = self.a
        self.a += 1
        return x

myclass = MyNumbers()
myiter = iter(myclass)

print(next(myiter))
print(next(myiter))
print(next(myiter))
print(next(myiter))
print(next(myiter))

1
2
3
4
5


In [5]:
# sampleList = [1,2,3,4,5,6,7,8]
sampleList = [i for i in range(1,9)]
sampleList2 = (i for i  in range(1,9))  #直接创建一个生成器
# print(list[sampleList])
print(type(sampleList), type(sampleList2))
print(sampleList2)
def sample():
    for i in range(1,10):
        yield i
generator1 = sample()
print(generator1)
for items in generator1:
    print(items)

<class 'list'> <class 'generator'>
<generator object <genexpr> at 0x000002C7C4ABB7B0>
<generator object sample at 0x000002C7C4ABB9E0>
1
2
3
4
5
6
7
8
9


In [1]:
 # 简单的生成器函数
def my_gen():
    n=1
    print("first")
    # yield区域
    yield n

    n+=1
    print("second")
    yield n

    n+=1
    print("third")
    yield n

a=my_gen()
print("next method:")
# 每次调用a的时候，函数都从之前保存的状态执行
print(next(a))
print(next(a))
print(next(a))


next method:
first
1
second
2
third
3


In [1]:
# 2)生成器

def func():
    yield 1
    yield 2

# 创建生成器对象（内部是根据生成器类generator创建的对象)，生成器类的内部也声明了:_iter_、_next_方法。
obj1=func()
v1=next(obj1)
print(v1)

v2=next(obj1)
print(v2)

v3=next(obj1)
print(v3)
    
# 如果按照迭代器的规定来看，其实生成器类面是一种特殊的迭代器类（生成器也是一个中特殊的迭代器)    

1
2


StopIteration: 

In [14]:
#3）可迭代对象

#如果一个类中有_iter_方法且返回一个迭代器对象﹔则我们称以这个类创建的对象为可迭代对象。
class Foo( object ):
    
    def _iter__(self):
        return 迭代器对象（生成器对象)

obj = Foo()#obj是可迭代对象。

#可迭代对象是可以使用for来进行循环，在循环的内部其实是先执行_iter_方法，获取其迭代器对象，然后再在内部执行这个迭代器对象的next功能，逐步取值。
for item in obj:
    pass

SyntaxError: invalid character in identifier (Temp/ipykernel_13352/2932171383.py, line 7)

In [15]:
# 4)一般可迭代器对象跟迭代器都是共同使用的
class IT():
    def __init__(self):
        self.counter=0

    def __iter__(self):
        return self

    def __next__(self):
        self.counter +=1
        if self.counter ==3:
            raise StopIteration()
        return self.counter
    
class Foo():
    def __iter__(self):
        return IT()
    
obj=Foo() #可迭代对象

for item in obj:  #循环可迭代对象时，内部先执行obj._iter_并获取迭代器对象;不断地执行迭代器对象的next方法。
    print(item)

1
2
